In [40]:
from headers import *

In [41]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

In [42]:
df = pd.read_excel("input_df.xlsx")
df.index.name = "index"
df.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
0,1,1,1,0,20,0,20,10,0
1,1,2,1,0,20,0,20,5,1
2,1,3,1,20,0,20,0,10,0
3,1,4,1,20,0,20,0,5,1
4,1,5,1,20,0,0,20,10,0
5,1,6,1,20,0,0,20,5,1
6,2,1,1,0,15,0,15,10,0
7,2,2,1,0,15,0,15,5,0
8,2,3,1,15,0,15,0,10,0


In [43]:
df['B_cr4first'] = [1 if x > 0 else 0 for x in df['now_ivt_4']]
df['B_cr3first'] = [1 if x > 0 else 0 for x in df['now_ivt_3']]
df['B_cr3second'] = [1 if x > 0 else 0 for x in df['wait_ivt_3']]
df['B_cr2second'] = [1 if x > 0 else 0 for x in df['wait_ivt_2']]
df['IVT_time'] = df[['now_ivt_4','now_ivt_3']].max(axis=1)
df_used = df[['pass_ID','scenario','time_crit','B_cr4first','B_cr3first','B_cr3second','B_cr2second','IVT_time','WT_time','output_choice']]
df_used.head(20)

,pass_ID,scenario,time_crit,B_cr4first,B_cr3first,B_cr3second,B_cr2second,IVT_time,WT_time,output_choice
index,,,,,,,,,,
0,1,1,1,0,1,0,1,20,10,0
1,1,2,1,0,1,0,1,20,5,1
2,1,3,1,1,0,1,0,20,10,0
3,1,4,1,1,0,1,0,20,5,1
4,1,5,1,1,0,0,1,20,10,0
5,1,6,1,1,0,0,1,20,5,1
6,2,1,1,0,1,0,1,15,10,0
7,2,2,1,0,1,0,1,15,5,0
8,2,3,1,1,0,1,0,15,10,0


## Model 01 - general model with ASC
### (analogous to (Preston et al., 2017))
### dep1 = [ASC_first]
### dep2 = [ASC_second] + [B_timecrit] * s1 + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [44]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [45]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_timecrit = Beta('BETA_timecrit',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = time_crit 
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_timecrit * s1 + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [46]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_007_01_general_model"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.754
BETA_CR2_second=	0.414
BETA_CR3_first=	-0.678
BETA_CR3_second=	0.34
BETA_CR4_first=	1.43
BETA_IVT=	0.0278
BETA_WT=	-0.275
BETA_timecrit=	-1.56
                    Value   Std err     t-test       p-value  Rob. Std err  \
ASC_SECOND       0.753754  0.095575   7.886519  3.108624e-15      0.097489   
BETA_CR2_second  0.414201  0.069792   5.934763  2.942705e-09      0.070205   
BETA_CR3_first  -0.677508  0.082887  -8.173895  2.220446e-16      0.083688   
BETA_CR3_second  0.339554  0.080529   4.216515  2.481063e-05      0.080872   
BETA_CR4_first   1.431263  0.081219  17.622196  0.000000e+00      0.081768   
BETA_IVT         0.027803  0.004338   6.409666  1.458387e-10      0.004869   
BETA_WT         -0.275226  0.020846 -13.203138  0.000000e+00      0.020810   
BETA_timecrit   -1.561388  0.105993 -14.731109  0.000000e+00      0.106166   

                 Rob. t-test  Rob. p-value  
ASC_SECOND          7.731661  1.065814e-14  
BETA_CR2_second     5.899871  3.637857e-09  
B

## Model 02 - model 01, time-critical trips only
### dep1 = [ASC_first]
### dep2 = [ASC_second] + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [47]:
# filter records
df_used_02 = df_used[df_used['time_crit']==1]

In [48]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used_02)

In [49]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [50]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_007_02_general_model_timecrit_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.129
BETA_CR2_second=	0.0924
BETA_CR3_first=	-0.942
BETA_CR3_second=	0.0363
BETA_CR4_first=	1.07
BETA_IVT=	0.0225
BETA_WT=	-0.297
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_SECOND       0.128755  0.132867  0.969050  3.325201e-01      0.130536   
BETA_CR2_second  0.092434  0.098621  0.937261  3.486243e-01      0.097986   
BETA_CR3_first  -0.941562  0.141551 -6.651771  2.895861e-11      0.139627   
BETA_CR3_second  0.036321  0.114598  0.316945  7.512854e-01      0.114031   
BETA_CR4_first   1.070317  0.116646  9.175797  0.000000e+00      0.115964   
BETA_IVT         0.022522  0.005813  3.874399  1.068883e-04      0.005752   
BETA_WT         -0.297456  0.031772 -9.362128  0.000000e+00      0.031943   

                 Rob. t-test  Rob. p-value  
ASC_SECOND          0.986360  3.239565e-01  
BETA_CR2_second     0.943334  3.455099e-01  
BETA_CR3_first     -6.743397  1.547251e-11  
BETA_CR3_second     0.318521  7.500898e-01  
BETA_CR4_first 

## Model 03 - model 01, non-time-critical trips only
### dep1 = [ASC_first]
### dep2 = [ASC_second] + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [51]:
# filter records
df_used_03 = df_used[df_used['time_crit']==0]

In [52]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used_03)

In [53]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [54]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_007_03_general_model_nontimecrit_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.615
BETA_CR2_second=	0.354
BETA_CR3_first=	-0.769
BETA_CR3_second=	0.261
BETA_CR4_first=	1.38
BETA_IVT=	0.0345
BETA_WT=	-0.259
                    Value   Std err     t-test       p-value  Rob. Std err  \
ASC_SECOND       0.614922  0.127561   4.820601  1.431267e-06      0.138543   
BETA_CR2_second  0.354140  0.095648   3.702534  2.134568e-04      0.098974   
BETA_CR3_first  -0.769387  0.106776  -7.205599  5.779821e-13      0.111834   
BETA_CR3_second  0.260781  0.111044   2.348447  1.885189e-02      0.113627   
BETA_CR4_first   1.384309  0.104583  13.236505  0.000000e+00      0.105964   
BETA_IVT         0.034539  0.006588   5.242274  1.586095e-07      0.008563   
BETA_WT         -0.259043  0.027720  -9.344877  0.000000e+00      0.027497   

                 Rob. t-test  Rob. p-value  
ASC_SECOND          4.438482  9.059535e-06  
BETA_CR2_second     3.578132  3.460590e-04  
BETA_CR3_first     -6.879749  5.995870e-12  
BETA_CR3_second     2.295065  2.172941e-02  
BETA_CR4_